# Emotion Classification using TAO EmotionNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080"> 

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained model and train an EmotionNet model on subset of CK+ dataset
* Run Inference on the trained model
* Export the retrained model to a .etlt file for deployment for DeepStream SDK

### Table of Contents

This notebook shows an example of emotion classification in the Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables, map drives, and install dependencies](#head-0)
1. [Install the TAO launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2) <br>
    2.1 [Verify downloaded dataset](#head-2-1) <br>
    2.2 [Convert dataset labels to required json format](#head-2-2) <br>
    2.3 [Verify dataset conversion](#head-2-3) <br>
    2.4 [Download pre-trained model](#head-2-4) <br>
3. [Generate tfrecords from labels in json format](#head-3)
4. [Provide training specification](#head-4)
5. [Run TAO training](#head-5)
6. [Evaluate trained models](#head-6)
7. [Run TAO inference](#head-7)
8. [Deploy](#head-8)


## 0. Set up env variables, map drives and install dependencies <a class="anchor" id="head-0"></a>
When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users' workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/emotionnet/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/emotionnet`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

In [1]:
!pwd

/home/skouchak/test/emotionnet


In [3]:
# Setting up env variables for cleaner command line commands.
import os

%env KEY=nvidia_tlt
%env NUM_GPUS=1
#%env USER_EXPERIMENT_DIR=/workspace/tao-experiments/emotionnet
#%env DATA_DOWNLOAD_DIR=/workspace/tao-experiments/emotionnet/orgData

# Set this path if you don't run the notebook from the samples directory.
%env NOTEBOOK_ROOT=/home/skouchak/test/emotionnet

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset is expected to be present in $LOCAL_PROJECT_DIR/emotionnet/orgData, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/emotionnet
# !PLEASE MAKE SURE TO UPDATE THIS PATH!.
%env LOCAL_PROJECT_DIR=/home/skouchak/test

# $SAMPLES_DIR is the path to the sample notebook folder and the dependency folder
# $SAMPLES_DIR/deps should exist for dependency installation
%env SAMPLES_DIR=/home/skouchak/test

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "emotionnet/orgData"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "emotionnet"
)

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_DATASET_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "dataset_specs"
)

#%env SPECS_DIR=/workspace/tao-experiments/emotionnet/specs
#%env DATASET_SPECS_DIR=/workspace/tao-experiments/emotionnet/dataset_specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR
!ls -rlt $LOCAL_DATASET_SPECS_DIR

env: KEY=nvidia_tlt
env: NUM_GPUS=1
env: NOTEBOOK_ROOT=/home/skouchak/test/emotionnet
env: LOCAL_PROJECT_DIR=/home/skouchak/test
env: SAMPLES_DIR=/home/skouchak/test
total 8
-rw------- 1 skouchak users 3376 Jun 15 19:23 emotionnet_tlt_pretrain.yaml
-rw------- 1 skouchak users 3282 Jun 15 19:23 emotionnet_tlt.yaml
total 4
-rw------- 1 skouchak users 455 Jun 15 19:23 dataio_config_ckplus.json


In [4]:
# Install requirement
!pip3 install -r $SAMPLES_DIR/deps/requirements-pip.txt

In [5]:
# Please replace fixme with the path to the wheel file that you downloaded from the developer zone link mentioned above.
# SKIP this step IF you have already installed the TAO launcher wheel.
!pip3 install nvidia-tao

In [6]:
# Initialize the TAO launcher
!tao info

Configuration of the TAO Toolkit Instance
dockers: ['nvidia/tao/tao-toolkit-tf', 'nvidia/tao/tao-toolkit-pyt', 'nvidia/tao/tao-toolkit-lm']
format_version: 2.0
toolkit_version: 3.22.05
published_date: 05/25/2022


## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

Please download the CK+ dataset from: https://www.pitt.edu/~emotion/ck-spread.htm.
You will need to sign the dataset user agreement and send it to the email provided on the agreement sheet to get access to the dataset.

After obtaining the dataset, please place the files in `$DATA_DOWNLOAD_DIR`. Please rename the dataset folder to `ckplus` as `+` sign may not be a valid folder name.
You will then have the following path for the CK+ dataset.
* Input data in `$LOCAL_DATA_DIR/ckplus`

You will then unzip the folder of ckplus dataset to the following folders.
* Image data: `$LOCAL_DATA_DIR/ckplus/cohn-kanade-images`
* Emotion label data: `$LOCAL_DATA_DIR/ckplus/Emotion`
* Landmarks label data: `$LOCAL_DATA_DIR/ckplus/Landmarks`

Note: please make sure that the folder name are as listed above. 

### A. Verify downloaded dataset <a class="anchor" id="head-2-1"></a>


In [ ]:
# Check the dataset is present
!mkdir -p $LOCAL_DATA_DIR
!if [ ! -d $LOCAL_DATA_DIR/ckplus/cohn-kanade-images ]; then echo 'Image Data folder not found, please download.'; else echo 'Found Image Data folder.';fi
!if [ ! -d $LOCAL_DATA_DIR/ckplus/Emotion ]; then echo 'Emotion labels folder not found, please download.'; else echo 'Found Emotion Labels folder.';fi
!if [ ! -d $LOCAL_DATA_DIR/ckplus/Landmarks ]; then echo 'Landmarks labels folder not found, please download.'; else echo 'Found Landmarks Labels folder.';fi

### B. Convert dataset labels to required json format <a class="anchor" id="head-2-2"></a>

In [ ]:
!python3 ckplus_convert.py --root_path $LOCAL_EXPERIMENT_DIR --dataset_folder_name ckplus --container_root_path $USER_EXPERIMENT_DIR

### C. Verify dataset conversion <a class="anchor" id="head-2-3"></a>

Please use the provided conversion script `ckplus_convert.py` to convert existing `Landmarks` and `Emotion` labels from `CK+` dataset to the required json label format. 

Note: for other public datasets, please use this script as a reference to convert the labels to required format. 

In [ ]:
# Sample json label.
!sed -n 1,201p $LOCAL_DATA_DIR/ckplus/data_factory/fiducial/S052_004_00000031_happy.json

### D. Download pre-trained model <a class="anchor" id="head-2-4"></a>

Please follow the instructions in the following to download and verify the pretrain model for emotionnet.

For EmotionNet pretrain model please download model: `nvidia/tao/emotionnet:trainable_v1.0`.

After downloading the pre-trained model, please place the files in `$LOCAL_EXPERIMENT_DIR/pretrain_models`
You will then have the following path

* pretrain model in `$LOCAL_EXPERIMENT_DIR/pretrain_models/emotionnet_vtrainable_v1.0/model.tlt`

In [7]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

env: CLI=ngccli_cat_linux.zip
--2022-07-12 22:24:13--  https://ngc.nvidia.com/downloads/ngccli_cat_linux.zip
Resolving ngc.nvidia.com (ngc.nvidia.com)... 13.32.164.118, 13.32.164.11, 13.32.164.19, ...
Connecting to ngc.nvidia.com (ngc.nvidia.com)|13.32.164.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33949316 (32M) [application/zip]
Saving to: ‘/home/skouchak/test/ngccli/ngccli_cat_linux.zip’

ngccli_cat_linux.zi 100%[===================>]  32.38M  42.3MB/s    in 0.8s    

2022-07-12 22:24:14 (42.3 MB/s) - ‘/home/skouchak/test/ngccli/ngccli_cat_linux.zip’ saved [33949316/33949316]

Archive:  /home/skouchak/test/ngccli/ngccli_cat_linux.zip
   creating: /home/skouchak/test/ngccli/ngc-cli/
   creating: /home/skouchak/test/ngccli/ngc-cli/wcwidth/
  inflating: /home/skouchak/test/ngccli/ngc-cli/wcwidth/version.json  
   creating: /home/skouchak/test/ngccli/ngc-cli/frozenlist/
  inflating: /home/skouchak/test/ngccli/ngc-cli/frozenlist/_frozenlist.cpython-39-x

In [8]:
# List models available in the model registry.
!ngc registry model list nvidia/tao/emotionnet:*

+-------+-------+-------+-------+-------+-------+------+-------+-------+
| Versi | Accur | Epoch | Batch | GPU   | Memor | File | Statu | Creat |
| on    | acy   | s     | Size  | Model | y Foo | Size | s     | ed    |
|       |       |       |       |       | tprin |      |       | Date  |
|       |       |       |       |       | t     |      |       |       |
+-------+-------+-------+-------+-------+-------+------+-------+-------+
| train | 91.0  | 100   | 1     | V100  | 4.4   | 4.38 | UPLOA | Aug   |
| able_ |       |       |       |       |       | MB   | D_COM | 19,   |
| v1.0  |       |       |       |       |       |      | PLETE | 2021  |
| deplo | 91.0  | 100   | 1     | V100  | 4.3   | 4.33 | UPLOA | Aug   |
| yable |       |       |       |       |       | MB   | D_COM | 19,   |
| _v1.0 |       |       |       |       |       |      | PLETE | 2021  |
+-------+-------+-------+-------+-------+-------+------+-------+-------+


In [9]:
# Create the target destination to download the model.
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrain_models/

In [10]:
# Download the pretrained model from NGC
!ngc registry model download-version nvidia/tao/emotionnet:trainable_v1.0 \
    --dest $LOCAL_EXPERIMENT_DIR/pretrain_models/

Downloaded 4.38 MB in 3s, Download speed: 1.46 MB/s               
----------------------------------------------------
Transfer id: emotionnet_vtrainable_v1.0 Download status: Completed.
Downloaded local path: /home/skouchak/test/emotionnet/pretrain_models/emotionnet_vtrainable_v1.0
Total files downloaded: 1 
Total downloaded size: 4.38 MB
Started at: 2022-07-12 22:25:04.022446
Completed at: 2022-07-12 22:25:07.027964
Duration taken: 3s
----------------------------------------------------


In [11]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/pretrain_models/emotionnet_vtrainable_v1.0

total 4484
-rw------- 1 skouchak users 4588024 Jul 12 22:25 model.tlt


In [13]:
# Check the dataset is present
!if [ ! -f $LOCAL_EXPERIMENT_DIR/pretrain_models/emotionnet_vtrainable_v1.0/model.tlt ]; then echo 'Pretrain model file not found, please download.'; else echo 'Found Pretrain model file.';fi

Found Pretrain model file.


## 3. Generate tfrecords from labels in json format <a class="anchor" id="head-3"></a>
* Create the tfrecords using the dataset_convert command


In [ ]:
!emotionnet dataset_convert -c $LOCAL_DATASET_SPECS_DIR/dataio_config_ckplus.json

In [ ]:
# Check the result folder is present
!mkdir -p $LOCAL_EXPERIMENT_DIR
!if [ ! -d $LOCAL_EXPERIMENT_DIR/postData/ckplus/Ground_Truth_DataFactory ]; then echo 'Ground truth folder not found.'; else echo 'Found Ground truth folder.';fi
!if [ ! -d $LOCAL_EXPERIMENT_DIR/postData/ckplus/GT_user_json ]; then echo 'GT user json folder not found.'; else echo 'Found GT user json folder.';fi

## 4. Provide training specification <a class="anchor" id="head-4"></a>
* Tfrecords for the train datasets
    * In order to use the newly generated tfrecords for training, update the 'ground_truth_folder_name' and 'tfrecords_directory_path' parameters of 'dataset_info' section in the spec file at `$LOCAL_SPECS_DIR/emotionnet_tlt_pretrain.yaml`
* Pre-trained model path
    * Update "pretrained_model_path" in the spec file at `$LOCAL_SPECS_DIR/emotionnet_tlt_pretrain.yaml`
    * If you want to training from random weights with your own data, you can enter "null" for "pretrained_model_path" section
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.

In [ ]:
!cat $LOCAL_SPECS_DIR/emotionnet_tlt_pretrain.yaml

## 5. Run TAO training <a class="anchor" id="head-5"></a>
* Provide the sample spec file and the output directory location for models

*Note: The training may take hours to complete. Also, the remaining notebook, assumes that the training was done in single-GPU mode. 

In [14]:
!emotionnet train

/bin/bash: emotionnet: command not found


In [ ]:
!emotionnet train -e $LOCAL_SPECS_DIR/emotionnet_tlt_pretrain.yaml \
                      -r $LOCAL_USER_EXPERIMENT_DIR/experiment_result/exp1 \
                      -k $KEY

In [ ]:
!ls -lh $LOCAL_EXPERIMENT_DIR/experiment_result

## 6. Evaluate the trained model <a class="anchor" id="head-6"></a>


In [ ]:
!emotionnet evaluate -m $LOCAL_USER_EXPERIMENT_DIR/experiment_result/exp1/model.tlt \
                         -r $LOCl_USER_EXPERIMENT_DIR/experiment_result/exp1 \
                         -e $LOCAL_SPECS_DIR/emotionnet_tlt_pretrain.yaml \
                         -k $KEY

In [ ]:
# check the Evaluation result file and summary file and is presented
!if [ ! -f $LOCAL_EXPERIMENT_DIR/experiment_result/exp1/eval_results.txt ]; then echo 'Evaluation result summary file not found, please generate.'; else echo 'Found Evaluation result summary file.';fi
!if [ ! -f $LOCAL_EXPERIMENT_DIR/experiment_result/exp1/full_results.txt ]; then echo 'Evaluation result file not found, please generate.'; else echo 'Found Evaluation result file.';fi

## 7. Visualize Inference <a class="anchor" id="head-7"></a>

In this section, we run the inference tool to generate inferences on the trained models.

In [ ]:
# Running inference for detection on n images
!emotionnet inference -e $LOCAL_SPECS_DIR/emotionnet_tlt_pretrain.yaml \
                          -i $LOCAL_DATA_DOWNLOAD_DIR/ckplus/data_factory/fiducial/S111_001_00000013_surprise.json \
                          -m $LOCAL_USER_EXPERIMENT_DIR/experiment_result/exp1/model.tlt \
                          -o $LOCAL_USER_EXPERIMENT_DIR \
                          -k $KEY 

In [ ]:
!sed -n 1,1p $LOCAL_EXPERIMENT_DIR/result.txt

In [ ]:
# Simple emotion inference overlay visualizer
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import os
import IPython.display
import PIL.Image


def visualize_images(output_path, num_images=10):
    root_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], output_path)
    result_path = os.path.join(root_path, 'result.txt')
    f = open(result_path, 'r')
    lines = f.readlines()
    
    for k in range(0, num_images):
        line = lines[k]
        content = line.split(' ')
        docker_data_path = os.environ['DATA_DOWNLOAD_DIR']
        print(docker_data_path)
        path_info = content[0].split(str(docker_data_path))
        image_path = os.environ['LOCAL_DATA_DIR'] + path_info[-1]
        emotion_label = content[1]
        print(image_path, emotion_label)
        img = cv2.imread(image_path, 0)
        cv2.putText(img, "Emotion: " + emotion_label, (40, 40), 0, 1, 255)
        IPython.display.display(PIL.Image.fromarray(img))

In [ ]:
# Visualizing the first 2 images.
OUTPUT_PATH = '' # relative path from $USER_EXPERIMENT_DIR.
IMAGES = 1
visualize_images(OUTPUT_PATH, num_images=IMAGES)

## 8. Deploy <a class="anchor" id="head-7"></a>

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_final
# Removing a pre-existing copy of the etlt if there has been any.
import os
output_file=os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'],
                         "experiment_dir_final/emotionnet_onnx.etlt")
if os.path.exists(output_file):
    os.system("rm {}".format(output_file))
!tao emotionnet export -m $USER_EXPERIMENT_DIR/experiment_result/exp1/model.tlt \
                       -o $USER_EXPERIMENT_DIR/experiment_dir_final/emotionnet_onnx.etlt \
                       -t tfonnx \
                       -k $KEY

In [ ]:
# check the Deployed file is presented
!if [ ! -f $LOCAL_EXPERIMENT_DIR/experiment_dir_final/emotionnet_onnx.etlt ]; then echo 'Deployed file not found, please generate.'; else echo 'Found Deployed file folder.';fi